In [3]:
# Custom style
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [161]:
import pandas as pd
import itertools
import numpy as np

In [76]:
# import os
# os.system("git add . && git commit -m 'solved for lists' && git push")

0

In [194]:
num_customerz = 10*24
time_periodz = 24*60
num_repz = 4

In [214]:
%%time
call_in_time_list =  [np.random.choice(range(1,time_periodz)) for i in range(num_customerz)]
call_in_time_list.sort()

call_duration_list = [5 for i in range(num_customerz)] #np.random.choice([1,2,3,4,5])
wait_times_list = []
next_available = [0 for x in range(num_repz)]
rep_busy = [[] for x in range(num_repz)]
rep_customers = [[] for x in range(num_repz)]


for time_entry in range(len(call_in_time_list)):
    index_of_prev_min = next_available.index(min(next_available))
    end_time = call_duration_list[time_entry] + max(min(next_available), call_in_time_list[time_entry])
    
    
    next_available[index_of_prev_min] = end_time
    rep_busy[index_of_prev_min].append([x for x in range(end_time - call_duration_list[time_entry], end_time)])
    wait_times_list.append( end_time - (call_duration_list[time_entry] + call_in_time_list[time_entry]))
    rep_customers[index_of_prev_min].append(time_entry)

#45 ms before this point w/ standard config
df_customers = pd.DataFrame([wait_times_list, call_in_time_list, call_duration_list], index=['wait_time', 'call_in', 'call_duration']).T
df_reps = pd.DataFrame([rep_busy, rep_customers], index = ['busy_times', 'assigned_customers'])
df_txn_log = pd.DataFrame(index=[x for x in range (time_periodz)], columns= ['rep_' + str(i) for i in range(1, num_repz+1)])
for i in range(len(df_txn_log.columns)):
    df_txn_log.iloc[:, i]  = [1 if x in list(itertools.chain.from_iterable(df_reps.at['busy_times', i])) else 0 for x in range(time_periodz)]
#print(next_available)
#print(wait_times_list)
#print(rep_busy)
# Calculate q size?

CPU times: user 162 ms, sys: 0 ns, total: 162 ms
Wall time: 159 ms


In [215]:
# deque for q stuff

In [216]:
# import seaborn as sns
# sns.heatmap(df_txn_log.values)

In [217]:
df_txn_log.head()

,rep_1,rep_2,rep_3,rep_4
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,1,0,0,0
4,1,0,0,0


In [218]:
df_txn_log.sum().sum() / sum(call_duration_list) 

0.9975

In [221]:
1 / (1-.9975)

400.0000000000085

In [224]:
df_txn_log.shape[0] * (1-.9975)

3.5999999999999233